In [2]:
# current submission is .74.
import pandas as pd
from sklearn.linear_model import LinearRegression

In [1]:
#!kaggle competitions download -c titanic
#!kaggle datasets download -d zynicide/wine-reviews # Use this to get data if you want it.

In [4]:
og_df = pd.read_csv('./data/train.csv', engine='python')

In [5]:
# just rounds the models prediction to 0 or 1.
def get_model_val(df_row):
    if(lin_reg.predict([df_row])[0] > .5):
        return 1
    else:
        return 0

def clean_data(df):
    # Transformations here.
    new_df = df.copy() # To reset the data so I can do this all in one cell.

    new_df['Age'] = new_df['Age'].fillna(new_df.Age.median())

    mf_vals = {'male': 0, 'female': 1}
    new_df['Sex'].replace(mf_vals, inplace=True)

    embarked_vals ={'S': 1, 'C':2, 'Q':3}
    new_df['Embarked'].replace(embarked_vals, inplace=True)
    new_df['Embarked'] = new_df['Embarked'].fillna(0)
    # drop some columns...
    new_df = new_df.drop(columns=['Name', 'Ticket', 'Cabin'])
    return new_df

In [6]:
df = clean_data(og_df)
df_labels = df['Survived'].copy()
df = df.drop(columns=['Survived'])


In [7]:
df

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,0,22.0,1,0,7.2500,1.0
1,2,1,1,38.0,1,0,71.2833,2.0
2,3,3,1,26.0,0,0,7.9250,1.0
3,4,1,1,35.0,1,0,53.1000,1.0
4,5,3,0,35.0,0,0,8.0500,1.0
...,...,...,...,...,...,...,...,...
886,887,2,0,27.0,0,0,13.0000,1.0
887,888,1,1,19.0,0,0,30.0000,1.0
888,889,3,1,28.0,1,2,23.4500,1.0
889,890,1,0,26.0,0,0,30.0000,2.0


In [8]:
lin_reg = LinearRegression()

In [9]:
lin_reg.fit(df,df_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [11]:
lin_reg.predict([df.iloc[1]])[0] # sanity check

0.9082032156874049

In [12]:
tr_og_df = pd.read_csv('./data/test.csv', engine='python')
# we also want to transform the data in this DF to make it easier to predict.

# We probably want to figure out how to do a relative path here.
#tr_og_df['Survived'] = df.apply(lambda row: get_model_val(row), axis=1)

In [13]:
tr_df = clean_data(tr_og_df)
tr_df['Fare'] = tr_df['Fare'].fillna(0)

In [14]:
tr_df['Survived'] = tr_df.apply(lambda row: get_model_val(row), axis=1)

In [15]:
final_df = tr_df[['PassengerId', 'Survived']].copy()

In [18]:
final_df.to_csv('output.csv', index=False)
# submit me via command line thanks.